## CS585: Natural Language Processing
### Morphology and Finite State Transducers
<br><br>
#### Illinois Institute of Technology  
#### Aron Culotta

<br><br><br><br><br>

## Last time

![../l01/figs/automata.png](../l01/figs/automata.png)

#### FSA

![../l01/figs/baa.png](../l01/figs/baa.png)

#### Regular expression

``` python
re.match('^baa+!$', text)
```

#### Regular Language

$$L(m) = \{baa!, baaa!, baaaa!, baaaaa!, ...\}$$

## Today

- Morphology
- Finite State Transducers
- Some more about projects, github, etc.

## Morphology

Study of the sub-word units of meaning

![../l01/figs/morph.png](../l01/figs/morph.png)


#### Simpler example

> *foxes* $\rightarrow$ fox +N +PL  

(foxes consists of the noun "fox" plus a plural ending)

![figs/morph_ex.png](figs/morph_ex.png)


<br><br><br>
### Morphology Jargon

**morpheme:** minimal unit of a language that has meaning
> **fox** has one morpheme (**fox**)  
> **cats** has two morphemes (**cat** and **-s**)
  
<br><br>
Two types of morphemes:
1. **stem:** "main" morpheme of the word; gives the central meaning
  > **cat** is the stem of **cats**
2. **affix:** provides additional meaning; or modifies the meaning of the stem.
  > **-s** is the affix of **cats**
<br><br>

Four types of affixes:
1. **prefix**: precede the stem
  > **un-** in **unbuckle**
2. **suffix**: follow the stem
  > **-s** in **cats**
3. **infix**: inside the stem
  > Rare in English: **abso-freakin-lutely**.  
  > In Spanish: **-ita** in **Christinita** ("little Christine")
4. **circumfix:** precede *and* follow the stem.
  > Rare in English
  > In German: **sagen** ("to say") becomes **gesagt** ("said"), using **ge-** and **-t**.

<br><br><br>
Two ways to create words from morphemes:

1. **inflection:** combining a stem with a morpheme to produce a word in the *same class* as the stem, typically serving a syntactic function
  > **-s** in **cats**
2. **derivation:** combining a stem with a morpheme to produce a word in a *different class* as the stem, with a possibly different meaning
  > **computer** $\rightarrow$ **computerize** $\rightarrow$ **computerization** ($N \rightarrow V \rightarrow N$)

## Morphological Parsing

Given a word, how can we decompose it into morphemes?

> **cats** $\rightarrow$ **cat +N +PL**  
> **caught** $\rightarrow$ **catch +V +PAST**

Convert a **morpheme** into a **stem** and morphological **features**.

Components of a mophological parser:
1. **Lexicon:** list of stems, their parts of speech, and affixes
2. **Morphotactics:** Rules determining the order of morphemes
  > In English, pluralization morphemes appear as a **suffix**, not a **prefix**
3. **Spelling:** Rules determining how morphemes change when combined
  > **city** $\rightarrow$ **cities** ; $y \rightarrow ie$

## FSAs to represent inflection

![figs/inflection.png](figs/inflection.png)

<br><br><br>

## FSAs to represent derivation

How can we write adjectives?


![figs/adj.png](figs/adj.png)

**Hypothesis**: adjectives consist of
- an optional prefix **un-**
- a required root (e.g., **cool**)
- an optional suffix (**-er**, **-est**, **-ly**)

<br><br><br>
![figs/ant1.png](figs/ant1.png)

What forms does this FSA allow that are not valid?
<br><br><br>

> unbig, redly

So, we need to divide adjectives into different groups depending on the type of affixes they can have.  
E.g., *adj-root*$_1$ and *adj-root*$_2$ below.

![figs/ant2.png](figs/ant2.png)

<br><br><br>

## Letter-level FSA

![figs/letter.png](figs/letter.png)

## From Recognition to Parsing

- The FSAs above allow us to say "this is a valid word or not" (**recognition**)
- **Parsing** requires us to explicitly identify stem, and features of the word.
> **cats** $\rightarrow$ **cat +N +PL**  

We need a function that maps letter sequences into morpheme and feature sequences.

We need a finite state machine that returns a string rather than a boolean.

This is called a **Finite State Transducer (FST)**.

We can visualize an FST as a two-tape automaton (one input, one output).

![figs/tapes.png](figs/tapes.png)

## Finite State Transducers

- Finite State Automata (FSAs):
  - define a formal language by defining a set of strings
    - **Input**: string
    - **Output**: valid or not

- Finite State Transducers (FSTs):
  - define a *relation* between sets of strings
    - **Input**: string
    - **Output**: string
    
    
    
We can still view an FST as a recognizer, but it is a recognizer of a **pair** of strings (input, output)
- Is this output a valid transformation of the input?

## FST, formally

- $Q$: a finite set of $N$ states $\{q_0 \ldots q_N\}$
- $\Sigma:$ a finite alphabet of complex symbols; each symbol consists of an input-output pair $o: i$
  - One symbol $i$ is from input alphabet $I$
  - One symbol $o$ is from the output alphabet $O$.
  - $\Sigma \subseteq O \times I$
  - $I$ and $O$ may contain the empty string $\epsilon$
- $q_0$: the start state
- $F$: the set of final states, $F \subseteq Q$
- $\delta(q,o: i)$: the transition function  between states. $Q \times \Sigma \rightarrow Q$
  - Given a state $q \in Q$ and a complex symbol  $o:i \in \Sigma$, return a new state $q' \in Q$.


Thus, FST accepts a language defined over *pairs* of symbols. E.g., in sheep speak:
- FSA: $\Sigma = \{b, a, !\}$
- FST: $\Sigma = \{a:a, b:b, !:!, a:!, a:\epsilon, \epsilon:!\}$

## FST for inflection

![figs/fst.png](figs/fst.png)

- $a:b$ means $b$ is read from the input tape and $a$ is written to the output tape
- $a:\epsilon$  means $a$ on output tape corresponds to nothing on the input tape (write without consuming another symbol)
- $a$ alone is a shortcut for $a:a$ (copy input to output)
- ^ indicates a morpheme boundary
- \# indicates word boundary

#### This can get big, fast

![figs/fstbig.png](figs/fstbig.png)

## Stemming

For some applications, we don't need the complete morphological parse, we just need to know that two words have the same stem.

E.g., instead of **caught** $\rightarrow$ **catch +V +PAST**  
we just need to know that **catch** and **caught** have the same stem.

This is useful in search engines, where **catch** may return results containing only the word **caught**.


#### Porter Stemmer

- Widely used system of cascading rules for stemming
> ATIONAL $\rightarrow$ ATE (e.g., relational $\rightarrow$ relate)  
> ING $\rightarrow \epsilon$ if stem contains a vowel (e.g., talking $\rightarrow$ talk) 

In [1]:
from nltk.stem import PorterStemmer # See nltk.org (`pip install nltk`)
# http://tartarus.org/~martin/PorterStemmer/
# Original paper: http://web.simmons.edu/~benoit/lis466/PorterStemmingAlgorithm.pdf
porter = PorterStemmer()
types = ['tied', 'ties', 'tis', 'bed', 'cities', 'city', 'talking', 'relational']
print(types)
[porter.stem(x) for x in types]

['tied', 'ties', 'tis', 'bed', 'cities', 'city', 'talking', 'relational']


['tie', 'tie', 'ti', 'bed', 'citi', 'citi', 'talk', 'relat']

In [2]:
types = ['bed', 'kiss',
         'tied', 'tis',
         'universal', 'university',
         'experiment', 'experience',
         'past', 'paste',
         'alumnus', 'alumni',
         'adhere', 'adhesion',
         'create', 'creation']
porter_results = [porter.stem(x) for x in types]
porter_results

['bed',
 'kiss',
 'tie',
 'ti',
 'univers',
 'univers',
 'experi',
 'experi',
 'past',
 'past',
 'alumnu',
 'alumni',
 'adher',
 'adhes',
 'creat',
 'creation']

In [3]:
from nltk.stem.wordnet import WordNetLemmatizer
# See description: https://wordnet.princeton.edu/wordnet/man/morphy.7WN.html
# Must download wordnet: >>> nltk.download('wordnet')
lemm = WordNetLemmatizer()
lemm_results = [lemm.lemmatize(x) for x in types]
print('%15s\t%15s\t%15s' % ('type', 'porter', 'lemmatizer'))
print('\n')
print('\n'.join(['%15s\t%15s\t%15s' % (t[0], t[1], t[2])
                 for t in zip(types, porter_results, lemm_results)]))

           type	         porter	     lemmatizer


            bed	            bed	            bed
           kiss	           kiss	           kiss
           tied	            tie	           tied
            tis	             ti	             ti
      universal	        univers	      universal
     university	        univers	     university
     experiment	         experi	     experiment
     experience	         experi	     experience
           past	           past	           past
          paste	           past	          paste
        alumnus	         alumnu	        alumnus
         alumni	         alumni	        alumnus
         adhere	          adher	         adhere
       adhesion	          adhes	       adhesion
         create	          creat	         create
       creation	       creation	       creation


In [4]:
porter.stem('women')

'women'

In [5]:
lemm.lemmatize('women')

'woman'

In [6]:
## Lemmatizers use part of speech
print(lemm.lemmatize('are', 'n'))
print(lemm.lemmatize('are', 'v'))
print(lemm.lemmatize('is', 'v'))

are
be
be


In [7]:
## Further reading at http://www.nltk.org/book/ch03.html

#### image sources

- https://www.cs.colorado.edu/~martin/SLP/

In [3]:
from IPython.core.display import HTML
HTML(open('../custom.css').read())